In [ ]:
import netCDF4 as nc
import numpy as np
import rioxarray
import xarray
import rasterio
from rasterio import features
from rasterio.plot import show
import geopandas as gpd
import os
# from osgeo import gdal
import datetime
import time

### Input definitions

In [ ]:
Res='8m'

WD = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/'
WD_INPUT = f'{WD}Inputmaps_{Res}/'
Sim_Domain = f'{WD_INPUT}SimDomain_{Res}.asc'

# Raw precipitation files. 
Path_Raw_Precip= f'{WD}Raw_Precipitation/KNMI_1km_EM_nc/RAD_NL25_RAC_MFBS_EM_5min_' # path without the timestamp and the fleformat at the end
File_format_Precip = '.nc'

# Ready input files for the model
Filename_Input_Precip = 'Precip_5min_KNMI_FilledWithMean_' # path without number or file format (whic will be .asc) 

NO_DATA = -9999

with rasterio.open(f'{Sim_Domain}') as proj_outline:
    kwargs = proj_outline.profile
    PROJECT_CRS = proj_outline.crs
    PROJECT_TRANSFORM = proj_outline.transform
    PROJECT_WIDTH = proj_outline.width
    PROJECT_HEIGHT = proj_outline.height
    PROJECT_SHAPE = proj_outline.shape
proj_outline.close()

### Creating list with time steps and definition of resampling function

In [ ]:
# Selecting the timespan for files
start_time = '202107150000'
end_time = '202107152355'

# Setting the timestep between precip files
timespan = 5

# Date for output file only
date = '13_140721'

#
timestep = datetime.timedelta(minutes = timespan)

start_time = datetime.datetime.strptime( start_time, '%Y%m%d%H%M' )
end_time = datetime.datetime.strptime( end_time, '%Y%m%d%H%M' )

t = start_time
time_list = []
while t <= end_time:
    time_str = datetime.datetime.strftime(t, '%Y%m%d%H%M')
    time_list.append(time_str)
    t += timestep
    
# print(time_list[0:4])
print(proj_outline)

#### Function definition for resampling

In the cell below, rioxarray is used instead of rasterio, because rasterio does not handle the .nc files properly. All files are opened and resampled to match the simulation domain. Subsequently, the nodata parts (the southern region of the catchment is not covered by the KNMI precipitation data) of the precipitation files are filled with the mean precipitation that fell in the complete simulation domain. Lastly the file is saved to an .asc file.

In [ ]:
def PrecipReprojResamp(Input_Prec, Output_Prec, Domain_To_Match):
    with rioxarray.open_rasterio(Input_Prec, 
                                 'r',
                                 mask_and_scale=True).rio.reproject_match(Domain_To_Match,  
                                                                           Resampling = 'nearest', 
                                                                           dtype = 'float32', 
                                                                           nodata = NO_DATA,
                                                                           ) as clipped:
        # filling nodata values with the mean
        array = clipped.data
        mean_prec = (np.mean(array[array > NO_DATA]))
        array[array == NO_DATA] = mean_prec      
        
        with rasterio.open( f'{Output_Prec}', 
                            'w',
                            **kwargs,
                            DECIMAL_PRECISION=3,
                            ) as dst:
                            dst.write(array)

#### Running the resampling file

The 'Num' variable can be adjusted to start numbering at a different number. For the RIM2D model, numbering starts at 1.

In [ ]:
DEM_match = rioxarray.open_rasterio(f'{Sim_Domain}')
Num = 1


tic = time.perf_counter()

for i in range(len(time_list)):
    nc_input = f'{Path_Raw_Precip}{time_list[i]}{File_format_Precip}'
    asc_output = f'{WD_INPUT}/rain/Rain_{date}/{Filename_Input_Precip}{date}_{Num}.asc'
    PrecipReprojResamp(nc_input, asc_output, DEM_match)
    Num = Num+1      

toc = time.perf_counter()
print(f'Time to tun = {toc-tic:0.4f} seconds')

### Calculating summed precip per day

In [ ]:
def precip_per_day(date, real_date, filenum):
    with rasterio.open(f'{WD_INPUT}rain/Rain_{date}/Precip_5min_KNMI_FilledWithMean_{date}_{filenum[0]}.asc') as dst_prec:
        print(f'{filenum[0]}')
        sum_prec = dst_prec.read(1)
        
        kwargs = dst_prec.profile
        
    for i in range(filenum[0]+1, filenum[1]):
        with rasterio.open(f'{WD_INPUT}rain/Rain_{date}/Precip_5min_KNMI_FilledWithMean_{date}_{i}.asc') as dst_prec:
            arr_prec = dst_prec.read(1)
            sum_prec = sum_prec + arr_prec
            print(i)
    
    show(sum_prec, cmap= 'Blues')
    
    with rasterio.open(f'{WD_INPUT}rain/Summed_rain_{real_date}.asc', 'w', **kwargs) as dst:
        dst.write(sum_prec, indexes = 1)      

In [ ]:
filerange = [217,289]

folder_date = '120721'
real_date = '12_07_21'
precip_per_day(folder_date, real_date, filerange)

### Part below is to rename files, ONLY RUN IF NEED TO ADJUST

The RIM2D model always starts with precipitation file number one and counts upwarts. So when running a new timespan or different days, there might be a need to adjust the filename sof exiting precipitation data instead of re-running the resampling code above. The method of renaming the files is namely a lot faster. However, there is a risk of making mistakes and mixing up files, so be carefull!

In [2]:
old_date = '160721'
new_date = '13_14_15_160721'

num=1
for i in range(865, 1153):
    old_name_asc = f'{WD_INPUT}rain/Rain_{old_date}/Precip_5min_KNMI_FilledWithMean_{old_date}_{num}.asc'
    old_name_prj = f'{WD_INPUT}rain/Rain_{old_date}/Precip_5min_KNMI_FilledWithMean_{old_date}_{num}.prj'
    num = num+1
#     print(old_name_asc)
    new_name_asc = f'{WD_INPUT}rain/Rain_{new_date}/Precip_5min_KNMI_FilledWithMean_{new_date}_{i}.asc'
    new_name_prj = f'{WD_INPUT}rain/Rain_{new_date}/Precip_5min_KNMI_FilledWithMean_{new_date}_{i}.prj'
    os.rename(old_name_asc, new_name_asc)
    os.rename(old_name_prj, new_name_prj)
    
#     print(new_name)